In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import numpy as np

fname = 'relu_1024-20-20-20_nbin=5'
#fname = 'tanh_1024-20-20-20_nbin=5'
import cPickle
d = cPickle.load(open('logs/'+fname, 'rb'))
ACTIVATION = d['ACTIVATION']
PLOT_LAYERS = d['PLOT_LAYERS']
saved_logs = d['LOGS']

print('Last epoch:', d['last_epoch'])

epochs = sorted(saved_logs.keys())


In [ ]:
DO_SAVE = False

pfx = fname 
#ctime = int(time.time())
#'%d_%s' % (ctime, ACTIVATION)

sns.set_style("white")

plt.figure(figsize=(11,12))
gs = gridspec.GridSpec(5,len(PLOT_LAYERS))
plt.subplot(gs[0,0])
plt.plot(epochs, [saved_logs[epoch]['trn_loss'] for epoch in epochs], label='Train')
plt.plot(epochs, [saved_logs[epoch]['tst_loss'] for epoch in epochs], label='Test')
plt.ylim([0, plt.ylim()[1]])
plt.ylabel('Cross entropy loss')
plt.legend()
plt.gca().set_xscale("log", nonposx='clip')
plt.title(ACTIVATION)


for lndx, layerid in enumerate(PLOT_LAYERS):
    t = 'tst'
    plt.subplot(gs[1,lndx])

    kde_kl = [saved_logs[epoch][t+'_layer_%d_h_upper' % lndx] for epoch in epochs]
    kde_bd = [saved_logs[epoch][t+'_layer_%d_h_lower' % lndx] for epoch in epochs]
    #plt.plot(epochs, kde_kl, 'r:', label="$H_{KL}$")
    #plt.plot(epochs, kde_bd, 'g:', label="$H_{BD}$")
    plt.fill_between(epochs, kde_bd, kde_kl, color='greenyellow')

    plt.title('%s - %s layer %d'%(ACTIVATION, t, layerid))
    plt.xlabel('Epochs')
    plt.ylabel('KDE Entropy')
    plt.gca().set_xscale("log", nonposx='clip')

for lndx, layerid in enumerate(PLOT_LAYERS):
    t = 'tst'
    plt.subplot(gs[2,lndx])
    
    cmeas = 'bin'
    plt.plot(epochs, [saved_logs[epoch][t+'_layer_%d_h_%s' %( lndx, cmeas)] for epoch in epochs], 'b', label="$H_{%s}$"%cmeas)
    #plt.plot(epochs, [saved_logs[epoch][t+'_layer_%d_h_binstd'% rowndx] for epoch in epochs], 'k', label="$H_{binstd}$")
    plt.ylim(0, 1.1*plt.ylim()[1])
    plt.title('%s - %s layer %d'%(ACTIVATION, t, layerid))
    plt.xlabel('Epochs')
    plt.ylabel('MI binned [-1,1]')
    plt.gca().set_xscale("log", nonposx='clip')
    
for lndx, layerid in enumerate(PLOT_LAYERS):
    t = 'tst'
    plt.subplot(gs[3,lndx])
    
    cmeas = 'binstd'
    plt.plot(epochs, [saved_logs[epoch][t+'_layer_%d_h_%s' %( lndx, cmeas)] for epoch in epochs], 'b', label="$H_{%s}$"%cmeas)
    #plt.plot(epochs, [saved_logs[epoch][t+'_layer_%d_h_binstd'% rowndx] for epoch in epochs], 'k', label="$H_{binstd}$")
    plt.ylim(0, 1.1*plt.ylim()[1])
    plt.title('%s - %s layer %d'%(ACTIVATION, t, layerid))
    plt.xlabel('Epochs')
    plt.ylabel('MI zscore binned')
    plt.gca().set_xscale("log", nonposx='clip')

for lndx, layerid in enumerate(PLOT_LAYERS):
    plt.subplot(gs[4,lndx])
    means = np.array([saved_logs[epoch]['trn_layer_%d_gradmean' % lndx] for epoch in epochs])
    stds  = np.array([saved_logs[epoch]['trn_layer_%d_gradstd' % lndx] for epoch in epochs])
    plt.plot(epochs, means, 'b', label="Mean")
    plt.plot(epochs, stds, 'orange', label="Std")
    plt.plot(epochs, means/stds, 'red', label="SNR")
    plt.plot(epochs, [saved_logs[epoch]['trn_layer_%d_weightsnorm' % lndx] for epoch in epochs], 'g', label="||W||")
    
    #plt.ylabel('Layer %d - SNR'%rowndx)
    #plt.subplot(gs[rowndx*2+1,colndx])
    #plt.plot(epochs, [saved_logs[epoch][t+'_layer_%d_weightsnorm' % rowndx] for epoch in epochs], 'b', label="Weight norm")
    #plt.ylabel('Layer %d - ||Weights||'%rowndx)
    #plt.xlabel('Epochs')
    plt.title('Layer %d'%layerid)
    plt.xlabel('Epoch')
    plt.gca().set_xscale("log", nonposx='clip')
    plt.gca().set_yscale("log", nonposy='clip')
    
plt.legend(loc='lower left')
plt.tight_layout()

if DO_SAVE:
    print("Saving")
    plt.savefig('plots/%s_summary.pdf'%pfx)

In [ ]:
plt.figure(figsize=(10,4))
for ndx, act in enumerate(['tanh','relu']):
    d = cPickle.load(open('logs/'+act+'_1024-20-20-20_nbin=5', 'rb'))
    ACTIVATION = d['ACTIVATION']
    PLOT_LAYERS = d['PLOT_LAYERS']

    epochs = sorted(saved_logs.keys())

    plt.subplot(1,2,ndx+1)
    for lndx, layerndx in enumerate(PLOT_LAYERS):
        vals = np.array([d['LOGS'][epoch]['tst_layer_%d_h_upper' % lndx] for epoch in epochs])
        vals -= lndx * 15 # np.min(vals)
        plt.plot(epochs, vals, label='Layer %d' % layerndx)
    plt.gca().set_xscale("log")

    plt.title(act)
plt.legend(loc='lower right')



In [ ]:
for epoch in epochs:
    print(np.log(10)-d['LOGS'][epoch]['tst_loss'])
    asdf

In [ ]:
import utils
trn, tst = utils.get_mnist()

In [ ]:
np.mean(tst.Y, axis=1)[1]

In [ ]:
adsfsdfdfasdfsd

In [ ]:
plt.figure(figsize=(15,5*len(PLOT_LAYERS)))
gs = gridspec.GridSpec(len(PLOT_LAYERS)+1, 2, )

for colndx, t in enumerate(['trn','tst']):
    plt.subplot(gs[rowndx,colndx])

    for rowndx, lndx in enumerate(PLOT_LAYERS):
        plt.plot(epochs, [saved_logs[epoch][t+'_layer_%d_h_lower' % rowndx] for epoch in epochs], 'g:', label="$H_{BD}$")

    plt.title('%s - %s layer %d'%(ACTIVATION, t,lndx))
    plt.xlabel('Epochs')
    plt.ylabel('Layer Activity Entropy')
    plt.gca().set_xscale("log", nonposx='clip')

    if t == 'tst':
        plt.legend(loc='lower right')
plt.tight_layout()
